In [1]:
import multinetx as mx
import numpy as np
import random
from ling_model import create_MultilayerGraph

思路：
首先求得上层和下层网络中每个节点度的排名向量，
其实，这个和pearson相关系数很像
## 先取从大到小的度值排序，从1开始

In [2]:
N = 8
mg = create_MultilayerGraph(N=N, p_g1=0.3, p_g2=0.4, seed_g1=28, seed_g2=25, intra_layer_edges_weight=2, inter_layer_edges_weight=3)

In [5]:
def get_degree_by_node(Graph, node, N, layer=0):
    return Graph.degree(N*layer + node) - 1
def get_degrees_by_layer(Graph, N, layer):
    degrees = []
    for i in range(N):
        degrees.append(get_degree_by_node(Graph, i, N, layer))
    return degrees

In [9]:
get_degrees_by_layer(mg, N, layer=0)

[4, 5, 4, 2, 5, 3, 3, 4]

In [35]:
set_degree_layer0 = set(get_degrees_by_layer(mg, N, layer=0))
set_degree_layer0

{2, 3, 4, 5}

In [15]:
sort_set_degree_layer0 = np.sort(list(set_degree_layer0))[::-1]
sort_set_degree_layer0

array([5, 4, 3, 2])

In [36]:
def get_sort_set_degree_layer(mg, N, layer):
    # 这里为了方便后面求度值排序，每个度都加上一
    degree_layer = [i+1 for i in get_degrees_by_layer(mg, N, layer)]
    set_degree_layer = set(degree_layer)
    sort_set_degree_layer = np.sort(list(set_degree_layer))[::-1]
    return sort_set_degree_layer

In [37]:
get_sort_set_degree_layer(mg, N, layer=0)

array([6, 5, 4, 3])

### 试试这个方法

In [23]:
dict(mg.degree)

{0: 5,
 1: 6,
 2: 5,
 3: 3,
 4: 6,
 5: 4,
 6: 4,
 7: 5,
 8: 4,
 9: 5,
 10: 1,
 11: 4,
 12: 4,
 13: 4,
 14: 3,
 15: 3}

In [27]:
sort_set_degree_layer0 = np.sort(list(set(list(dict(mg.degree).values())[0:N])))[::-1]
sort_set_degree_layer0

array([6, 5, 4, 3])

In [28]:
sort_set_degree_layer1 = np.sort(list(set(list(dict(mg.degree).values())[N:])))[::-1]
sort_set_degree_layer1 

array([5, 4, 3, 1])

In [30]:
def get_sort_set_degree_layer_new(mg, N, layer):
    set_degree_layer = set(list(dict(mg.degree).values())[layer*N:N*(layer+1)])
    sort_set_degree_layer = np.sort(list(set_degree_layer))[::-1]
    return sort_set_degree_layer

In [32]:
get_sort_set_degree_layer_new(mg, N, 1)

array([5, 4, 3, 1])

> 还是第一种方法简单

求分子的第一个参数

In [38]:
sort_set_degree_layer0 = get_sort_set_degree_layer(mg, N, layer=0)
sort_set_degree_layer1 = get_sort_set_degree_layer(mg, N, layer=1)

In [51]:
def get_sort_degree_byNode(mg, N, layer):
    sequ = []    
    for i in range(N):
        # 求该layer层节点i的度值
        if layer == 0:
            node_degree = mg.degree(i)
        elif layer == 1:
            node_degree = mg.degree(i+N)
        # 求该节点i在度的排序
        sort_set_degree_layer = get_sort_set_degree_layer(mg, N, layer)
        for j in range(sort_set_degree_layer.size):
            if node_degree == sort_set_degree_layer[j]:
                sequ.append(j+1)
                break
    return sequ

In [65]:
array1 = np.array(get_sort_degree_byNode(mg, N, layer=0))
array2 = np.array(get_sort_degree_byNode(mg, N, layer=1))

In [66]:
array1

array([2, 1, 2, 4, 1, 3, 3, 2])

In [67]:
array2

array([2, 1, 4, 2, 2, 2, 3, 3])

In [68]:
def get_avg_degree_product(array1, array2, N):
    return np.sum(array1*array2)/N

In [69]:
get_avg_degree_product(array1, array2, N)

5.5

分子的第二个参数

In [70]:
def get_avg_degree(array, N):
    return np.sum(array)/N

In [74]:
get_avg_degree(array2, N)

2.375

分子的结果

In [72]:
molecular = get_avg_degree_product(array1, array2, N) - get_avg_degree(array1, N) * get_avg_degree(array2, N)

In [73]:
molecular

0.15625

求分母

In [75]:
def get_deominator_one(array, N):
    a1 = get_avg_degree_product(array, array, N)
    a2 = np.square(get_avg_degree(array, N))
    return np.sqrt(a1 - a2)

In [76]:
get_deominator_one(array1, N)

0.9682458365518543

分母的结果

In [77]:
denominator = get_deominator_one(array1, N) * get_deominator_one(array2, N)

In [78]:
denominator

0.8297448779594846

式子的最后结果

In [79]:
molecular / denominator

0.18831089428867737

最后计算Spearson的函数

In [81]:
def get_degree_by_node(Graph, node, N, layer=0):
    return Graph.degree(N*layer + node) - 1
def get_degrees_by_layer(Graph, N, layer):
    degrees = []
    for i in range(N):
        degrees.append(get_degree_by_node(Graph, i, N, layer))
    return degrees
def get_avg_degree_product(array1, array2, N):
    return np.sum(array1*array2)/N
def get_avg_degree(array, N):
    return np.sum(array)/N
def get_deominator_one(array, N):
    a1 = get_avg_degree_product(array, array, N)
    a2 = np.square(get_avg_degree(array, N))
    return np.sqrt(a1 - a2)


def get_sort_set_degree_layer(mg, N, layer):
    # 这里为了方便后面求度值排序，每个度都加上一
    degree_layer = [i+1 for i in get_degrees_by_layer(mg, N, layer)]
    set_degree_layer = set(degree_layer)
    sort_set_degree_layer = np.sort(list(set_degree_layer))[::-1]
    return sort_set_degree_layer

def get_sort_degree_byNode(mg, N, layer):
    sequ = []    
    for i in range(N):
        # 求该layer层节点i的度值
        if layer == 0:
            node_degree = mg.degree(i)
        elif layer == 1:
            node_degree = mg.degree(i+N)
        # 求该节点i在度的排序
        sort_set_degree_layer = get_sort_set_degree_layer(mg, N, layer)
        for j in range(sort_set_degree_layer.size):
            if node_degree == sort_set_degree_layer[j]:
                sequ.append(j+1)
                break
    return sequ

def spearman_correlation_coefficient(Graph, N):
    # 首先判断网络是否是符合要求的网络
    assert mx.is_connected(Graph) == True, \
        "The network must be reachable.提示：遇到这种问题，一般是生成的随机化网络不满足条件，解决办法是重新运行或者在创建双层网络时增加边出现的概率。"
    assert Graph.is_directed() == False, \
        "It is only allowed for undirected complex network."
    assert len(Graph) > 3, \
        "Graph has less than three nodes."
    # 首先得到两个网络中各节点的排名
    array1 = np.array(get_sort_degree_byNode(mg, N, layer=0))
    array2 = np.array(get_sort_degree_byNode(mg, N, layer=1))
    # 求分子
    molecular = get_avg_degree_product(array1, array2, N) - get_avg_degree(array1, N) * get_avg_degree(array2, N)
    # 求分母
    denominator = get_deominator_one(array1, N) * get_deominator_one(array2, N)
    # 求Spearman相关系数
    return molecular / denominator

In [82]:
spearman_correlation_coefficient(mg, N)

0.18831089428867737